<a href="https://colab.research.google.com/github/Pavun-KumarCH/Agentic-RAG-Systems/blob/main/Llama_Index_ReAct_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Roadmap

Part-4 This video focus:

- Function Calling Agents + Agent Runner
- Agentic RAG
- Project - ReACT Agent: Building a Search Assistant Agent


In [4]:
#@title requirements
%pip install --q llama-index duckduckgo-search
%pip install --q llama-index-llms-mistralai llama-index-embeddings-mistralai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.0/260.0 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.1 MB/s eta 0:00:00


In [10]:
#@title Function Calling
import nest_asyncio, os
from duckduckgo_search import DDGS
from llama_index.core.tools import FunctionTool
from llama_index.llms.mistralai import MistralAI
from llama_index.embeddings.mistralai import MistralAIEmbedding # Agentic RAG

nest_asyncio.apply()

In [54]:
#@title Creating function tools for search

def search(query:str) -> str:
  """
  Args:
      query: user prompt
  return:
  context (str): search results to the user query
  """

  req = DDGS()
  response = req.text(query, max_results =4)
  context = ""
  for result in response:
    context += result['body']
  return context

search_tool = FunctionTool.from_defaults(fn = search)

In [11]:
#@title Setup the Model
from google.colab import userdata
from llama_index.core import Settings

mistral_api = userdata.get("MISTRALAI_API_KEY")
llm = MistralAI(model = "mistral-large-latest", api_key = mistral_api)
embed_model = MistralAIEmbedding(model = "mistral-embed", api_key = mistral_api)

Settings.llm = llm
Settings.embed_model = embed_model

In [20]:
#@title testing llm-api
llm.complete("tell me about AI ?").text

'Artificial Intelligence (AI) is a broad field of computer science dedicated to creating systems that can perform tasks that typically require human intelligence. These tasks include learning, reasoning, problem-solving, perception, and language understanding. AI can be categorized into several types, each with its own set of capabilities and applications:\n\n### Types of AI\n\n1. **Narrow AI (Weak AI)**:\n   - Designed to perform a narrow task (e.g., facial recognition, internet searches, or driving a car).\n   - Examples include Siri, Alexa, and self-driving cars.\n\n2. **General AI (Strong AI)**:\n   - Hypothetical AI that possesses the ability to understand, learn, and apply knowledge across a wide range of tasks at a level equal to or beyond human capabilities.\n   - As of now, General AI does not exist.\n\n3. **Superintelligent AI**:\n   - A hypothetical AI that surpasses human intelligence in every economically valuable work.\n   - This is a conceptual idea and not yet realized.

In [16]:
query = "Who was the Man of the Series in India vs Bangalesh 2024 Test Match ?"

In [21]:
function_call = llm.predict_and_call(tools = [search_tool],
                                     user_msg = query,
                                     allow_parallel_tool_calls = True)


In [18]:
type(function_call)

llama_index.core.chat_engine.types.AgentChatResponse

In [19]:
function_call.response

"IND vs BDESH 2024/25, India vs Bangladesh Match Report: Bangladesh captain Najmul Hossain Shanto's 82 kept the bowling at bay, but Ashwin and Ravindra Jadeja shared nine wicketsThe 312 balls India batted for in this Test is the second-fewest they have faced - and the fourth-fewest overall - to win a Test match, behind the 281 against South Africa in Cape Town earlier ...Ravichandran Ashwin 's impressive performance with both bat and ball in the two-match India vs Bangladesh Test 2024 series won him the Player of the Series award. The all-rounder had a tally of 11 wickets and 114 runs from the two matches that saw him win the coveted award for the 11th time in his Test career.India vs Bangladesh test series 2024 will start on September 19. The Indian cricket team will begin their 2024-25 home season with a two-match test series against Bangladesh.It will be India's first test series at home since their victory over England earlier this year."

# Second Approach for Function Calling Agent
 * Function Calling Agent Worker

In [22]:
from llama_index.core.agent import FunctionCallingAgent, FunctionCallingAgentWorker

In [23]:
agent = FunctionCallingAgent.from_tools(tools = [search_tool],
                                        llm = llm,
                                        verbose = True,
                                        allow_parallel_tool_calls = True)

In [24]:
response = agent.chat(query)

> Running step d0d9066e-cf64-4f7a-bae8-f620eb233940. Step input: Who was the Man of the Series in India vs Bangalesh 2024 Test Match ?
Added user message to memory: Who was the Man of the Series in India vs Bangalesh 2024 Test Match ?
=== Calling Function ===
Calling function: search with args: {"query": "Man of the Series in India vs Bangladesh 2024 Test Match"}
=== Function Output ===
The 312 balls India batted for in this Test is the second-fewest they have faced - and the fourth-fewest overall - to win a Test match, behind the 281 against South Africa in Cape Town earlier ...Ashwin scored a hundred and took six wickets, while Jadeja chipped in with three, to secure a 1-0 lead for India in Chennai. Shanto made 82 for Bangladesh, but could not prevent the defeat.Ravichandran Ashwin 's impressive performance with both bat and ball in the two-match India vs Bangladesh Test 2024 series won him the Player of the Series award. The all-rounder had a tally of 11 wickets and 114 runs from th

In [25]:
print(response)

Ravichandran Ashwin was named the Man of the Series in the India vs Bangladesh 2024 Test Match. He had an impressive performance with both bat and ball, scoring 114 runs and taking 11 wickets across the two matches. This was his 11th Player of the Series award in his Test career.


# Third Approach and the Recommended one : Agent Runner

In [26]:
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    tools = [search_tool],
    llm = llm,
    verbose = True,
    allow_parallel_tool_calls = True
)

agent_runner = AgentRunner(agent_worker)

In [28]:
runner_response = agent_runner.query(query)

Added user message to memory: Who was the Man of the Series in India vs Bangalesh 2024 Test Match ?
=== Calling Function ===
Calling function: search with args: {"query": "Man of the Series in India vs Bangladesh 2024 Test Match"}
=== Function Output ===
The 312 balls India batted for in this Test is the second-fewest they have faced - and the fourth-fewest overall - to win a Test match, behind the 281 against South Africa in Cape Town earlier ...IND vs BDESH 2024/25, India vs Bangladesh Match Report: Bangladesh captain Najmul Hossain Shanto's 82 kept the bowling at bay, but Ashwin and Ravindra Jadeja shared nine wicketsRavichandran Ashwin 's impressive performance with both bat and ball in the two-match India vs Bangladesh Test 2024 series won him the Player of the Series award. The all-rounder had a tally of 11 wickets and 114 runs from the two matches that saw him win the coveted award for the 11th time in his Test career.Get cricket scorecard of 1st Test, IND vs BAN, Bangladesh in 

In [29]:
runner_response.response

'Ravichandran Ashwin was named the Man of the Series in the India vs Bangladesh 2024 Test Match. His impressive performance with both bat and ball, including a tally of 11 wickets and 114 runs from the two matches, earned him the coveted award for the 11th time in his Test career.'

In [30]:
# you can also use chat to get response
runner_chat = agent_runner.chat(query)

Added user message to memory: Who was the Man of the Series in India vs Bangalesh 2024 Test Match ?
=== Calling Function ===
Calling function: search with args: {"query": "Man of the Series in India vs Bangladesh 2024 Test Match"}
=== Function Output ===
The 312 balls India batted for in this Test is the second-fewest they have faced - and the fourth-fewest overall - to win a Test match, behind the 281 against South Africa in Cape Town earlier ...IND vs BDESH 2024/25, India vs Bangladesh Match Report: Bangladesh captain Najmul Hossain Shanto's 82 kept the bowling at bay, but Ashwin and Ravindra Jadeja shared nine wicketsIndia vs Bangladesh test series 2024 will start on September 19. The Indian cricket team will begin their 2024-25 home season with a two-match test series against Bangladesh.It will be India's first test series at home since their victory over England earlier this year.World No. 1 Test bowler Ravichandran Ashwin equalled legendary Sri Lankan spinner Muttiah Muralitharan

In [31]:
runner_chat.response

'Ravichandran Ashwin was named the Man of the Series in the India vs Bangladesh 2024 Test Match. His impressive performance with both bat and ball, including a tally of 11 wickets and 114 runs from the two matches, earned him the coveted award for the 11th time in his Test career.'

# Agentic RAG


In [36]:
#@title Download Pdf data
!mkdir -p 'data/'
!wget 'https://arxiv.org/pdf/2409.16160.pdf'


--2024-12-20 10:59:25--  https://arxiv.org/pdf/2409.16160.pdf
Resolving arxiv.org (arxiv.org)... 151.101.3.42, 151.101.131.42, 151.101.67.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.3.42|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://arxiv.org/pdf/2409.16160 [following]
--2024-12-20 10:59:26--  http://arxiv.org/pdf/2409.16160
Connecting to arxiv.org (arxiv.org)|151.101.3.42|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21704565 (21M) [application/pdf]
Saving to: ‘2409.16160.pdf’

2409.16160.pdf      100%[===================>]  20.70M  --.-KB/s    in 0.1s    

2024-12-20 10:59:26 (199 MB/s) - ‘2409.16160.pdf’ saved [21704565/21704565]



In [35]:
#@title Load the documents
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(input_files = ["/content/data/2409.16160.pdf"]).load_data()

In [37]:
#@title Text Splitter in chunks
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size = 1024)
nodes = splitter.get_nodes_from_documents(documents)

In [39]:
#@title Indexing Ingestion
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

In [40]:
#@title create query engines
summary_query_engine = summary_index.as_query_engine(response_mode = "tree_summarize",
use_async = True)
vector_query_engine = vector_index.as_query_engine()

In [41]:
from llama_index.core.tools import QueryEngineTool
from llama_index.core.selectors import LLMSingleSelector
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine

summary_tool = QueryEngineTool.from_defaults(
    query_engine = summary_query_engine,
    description = ("This Agent is useful to summarize the Research paper in a simplied way by giving intuitions")
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine = vector_query_engine,
    description = (
        "This Agent is useful to answer to the user queries within the paper and retrieve the relevant piece of information"
    ),
)

In [42]:
query_engine = RouterQueryEngine(
    selector = LLMSingleSelector.from_defaults(),
    query_engine_tools = [summary_tool, vector_tool],
    verbose = True)

In [43]:
response = query_engine.query("give me technical architecture details in the paper")

Selecting query engine 1: The question asks for specific details within the paper, which aligns with the capability of retrieving relevant pieces of information..


In [44]:
print(response)

The technical architecture presented in the paper involves a framework that decomposes a video clip into three spatial components: the main human, the underlying scene, and floating occlusions. These components are organized in hierarchical layers based on 3D depth. The human component is further disentangled into identity and motion properties using canonical appearance transfer and structured body codes. These properties are then encoded into identity code (Cid) and motion code (Cmo). The scene and occlusion components are embedded with a shared Variational Autoencoder (VAE) encoder and reorganized as a full scene code (Cso).

These latent codes (Cid, Cmo, and Cso) are inserted into a diffusion-based decoder as conditions for video reconstruction. This architecture allows for controllable synthesis of various attributes and 3D-aware composition of the main object, foreground, and background. The network enables flexible user controls and handles complex 3D motions and natural object 

In [46]:
response_2 = query_engine.query("summarize the methodology from the paper")

Selecting query engine 0: The question asks to summarize a specific part of the paper, and choice (1) mentions that the agent is useful to summarize the research paper in a simplified way by giving intuitions..


In [47]:
print(response_2)

The methodology presented in the paper introduces a novel framework called MIMO for controllable character video synthesis. The core idea is to decompose a 2D video into spatial components considering the inherent 3D nature of video occurrence. Here's a summarized breakdown of the methodology:

1. **Spatial Decomposition**: The video is split into three main components: the main human subject, the underlying scene, and floating occlusions (foreground objects). This is achieved by lifting 2D pixels into 3D using monocular depth estimators and then decomposing the video clip into hierarchical layers based on 3D depth.

2. **Encoding Spatial Components**:
   - **Human Component**: This is further disentangled into identity and motion codes. The identity is captured using a canonical appearance transfer, while the motion is encoded via structured body codes anchored to a deformable human body model (SMPL).
   - **Scene and Occlusion Components**: These are embedded using a shared VAE encod

# ReAct Agent
- Build a simple project Search Assistant Agent.

In [48]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(tools = [search_tool],
                              llm = llm,
                              verbose = True,
                              allow_parallel_tool_call = True)

In [57]:
response = agent.chat("Tell me about Elon Musk ?")

> Running step bf125442-517c-4d6b-9a0b-1fcf25f62c0c. Step input: Tell me about Elon Musk ?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: search
Action Input: {'query': 'Elon Musk'}
Observation: Elon Reeve Musk was born on June 28, 1971, in Pretoria, South Africa's administrative capital. [3] [4] He is of British and Pennsylvania Dutch ancestry.[5] [6] His mother, Maye (née Haldeman), is a model and dietitian born in Saskatchewan, Canada, and raised in South Africa.[7] [8] [9] His father, Errol Musk, is a South African electromechanical engineer, pilot, sailor, consultant, emerald ...Elon Musk is the founder of SpaceX and Tesla Motors, among other companies. Their success has propelled him to become the richest person in the world, with a net worth greater than $450 billion.Elon Musk's role in taking down a bipartisan budget deal shows his ascendance as a political force. The social media warnings from the world's wea

In [58]:
print(response)

Elon Reeve Musk was born on June 28, 1971, in Pretoria, South Africa. He is of British and Pennsylvania Dutch ancestry. His mother, Maye (née Haldeman), is a model and dietitian born in Saskatchewan, Canada, and raised in South Africa. His father, Errol Musk, is a South African electromechanical engineer, pilot, sailor, consultant, and emerald miner.

Elon Musk is the founder of SpaceX and Tesla Motors, among other companies. Their success has propelled him to become the richest person in the world, with a net worth greater than $450 billion.

Elon Musk's role in taking down a bipartisan budget deal shows his ascendance as a political force. The social media warnings from the world's wealthiest man preceded President-elect Donald Trump's condemnation of the measure negotiated by GOP House Speaker Mike Johnson. Elon Musk made more than 150 posts about the bill on X, the social media site he owns.
